In [7]:
def scrap_alldata (self):
    pages_to_scrape = [
            
            '',
            '/trader/bestselling',
            '/trader/mostpopular',
            '/trader/categories/1',
            '/trader/categories/2',
            '/trader/categories/3',
            '/trader/categories/4',
            '/trader/categories/5',
            '/trader/categories/6',
            '/trader/categories/7',
            '/trader/categories/8',
            '/trader/newarrival'
        ]
    for page in pages_to_scrape:
        """Scrape the bestselling products page"""
        url = f"https://octopios.com/{page}"
        soup = self.get_page_content(url)
        
        if not soup:
            print(f"Failed to load page: {url}")
            continue

        products = self.extract_products(soup)
        self.all_scraped_data[page] = products
        
        products = []
        
        # Try different common product container selectors
        product_selectors = [
            '.product-item',
            '.product-card',
            '.product',
            '[class*="product"]',
            '.item',
            '.card',
            '[data-product]'
        ]
        
        product_elements = []
        for selector in product_selectors:
            elements = soup.select(selector)
            if elements:
                product_elements = elements
                print(f"Found {len(elements)} products using selector: {selector}")
                break
        
        # If no specific selectors work, try to find products by common patterns
        if not product_elements:
            # Look for divs that might contain product info
            all_divs = soup.find_all('div')
            product_elements = [div for div in all_divs if 
                             div.find('img') and 
                             (div.find(string=re.compile(r'\$|\d+\.\d+')) or 
                              div.find(['span', 'div'], class_=re.compile(r'price', re.I)))]
        
        print(f"Processing {len(product_elements)} potential product elements...")
        
        for element in product_elements:
            product_data = self.extract_product_info(element)
            if product_data['name']:  # Only add if we found a name
                products.append(product_data)
        
        return products
     
      